In [1]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! Nice to meet you.\n\nI'm an AI, and I'm here to help. How can I assist you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--07bae573-670d-422d-8d39-c1608d5d2ffc-0', usage_metadata={'input_tokens': 7, 'output_tokens': 384, 'total_tokens': 391, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 354}})

In [3]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As an AI, I don't have access to personal information about you, including your name. My purpose is to assist with information and tasks based on our current conversation, not to know your identity.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4ef44ee2-ca22-4bce-9862-eb2690355be7-0', usage_metadata={'input_tokens': 7, 'output_tokens': 517, 'total_tokens': 524, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 476}})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4d8de8e5-85b3-4923-9cb4-725f65693563-0', usage_metadata={'input_tokens': 25, 'output_tokens': 43, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 38}})

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)